In [1]:
import datetime
import os

import pandas as pd
import seaborn as sns


**Run the cells below to download the collisions data**

In [2]:
if not os.path.exists('data'):
    os.makedirs('data')

In [ ]:
!wget -nc -O data/collisions.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD

--2020-06-22 19:47:37--  https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data/collisions.csv’

data/collisions.csv     [     <=>            ]  96.53M  1.83MB/s               

# Data Science: First Steps

## The problem: planning deliveries in NYC

For our example data science problem, let's imagine we've been hired by a new courier service operating in the New York City area. The company needs to make deliveries efficiently, so the data science team is tasked with building data-driven tools to assist with operations.

For all intents and purposes, let's say we've simply been asked the following question: "How can we better plan deliveries?"

If this seems vague, that's the idea! Data scientists often need to turn general problems into more specific subproblems and then look for opportunities for quantitative insights within those subproblems.

### How does this differ from a typical directions application?
* Plan many deliveries at once
* Fixed number of packages to deliver in one day

### What makes a good route?
* Closely packed deliveries
* Fast
* Reliable

### Goal: towards proof-of-concept
Obviously, we can't jump right into building machine learning models and sophisticated route-planning software. In this notebook, we'll get the process started with some basic data exploration and descriptive analysis. We simply want to start getting familiar with the available data, discover some meaningful questions that the data can help us solve, and find some usable signal that we can apply towards modeling later on. 

## Acquiring data



### Road map
https://data.cityofnewyork.us/City-Government/road/svwp-sbcd

### Traffic
**Historical Data (2014-2018)**: https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts-2014-2018-/ertz-hr4r

**Real-time API**: https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq

### Collisions
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95



## Explore Collisions Data
There is plenty of data to dive into, as we see above, but we'll stick with the collisions data for now.

In [ ]:
df_all = pd.read_csv('data/collisions.csv',
                    low_memory=False)

In [ ]:
df = df_all[df_all['BOROUGH'] == 'MANHATTAN'].copy()
del df_all
df.head(1)

### New column: Timestamp
The dataset gives us a `CRASH DATE` AND A `CRASH TIME` column, but to make the most of Pandas' datetime functionality, we will combine these into a new column and convert it to datetime type.

In [ ]:
df['TIMESTAMP'] = df['CRASH DATE'] + ' ' + df['CRASH TIME']

In [ ]:
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

df.head(2)

In [ ]:
def make_timestamp_column(df):
    
    timestamp = df['CRASH DATE'] + ' ' + df['CRASH TIME']
    return pd.to_datetime(df['TIMESTAMP'])

In [ ]:
df['TIMESTAMP'] = make_timestamp_column(df)

### Check for missing values

In [ ]:
df.isna().head()

**Get the proportion of missing values per column**

In [ ]:
df.isna().mean().head()

In [ ]:
df.isna().mean().sort_values(ascending=False)

### Filtering:  collisions during delivery hours
Since we're conducting analysis for a courier company, we don't need to analyze data for collisions happening at all hours of the day. Let's make a reasonable assumption: deliveries will be made between 8 AM and 8 PM, so we'll restrict our analysis to that time period.

In [ ]:
def filter_delivery_hours(df, start=8, stop=20):
    df['HOUR'] = df['TIMESTAMP'].dt.hour
    df = df[(df['HOUR'] >= 8) & (df['HOUR'] <= 20)]
    return df

In [ ]:
df = filter_delivery_hours(df)
df[['HOUR']].plot.hist()

### Intersections:  `ON STREET` and `CROSS STREET`

We could make use of the `LOCATION` column to place collisions on a map, but the data gives us relevant information regarding streets and intersections as well. This is meaningful to us because we have to make street-level decisions in planning routes.

For instance, are there certain stretches of street we should avoid? We can start to address this question by creating a new column for `INTERSECTION`. 

**Missing values in at least one column**

In [ ]:
def make_intersections(df):
    intersections = (df['ON STREET NAME'].str.strip() 
                      + ' / '
                      + df['CROSS STREET NAME'].str.strip())
        
    return intersections

In [ ]:
df['INTERSECTION'] = make_intersections(df)
df_intersection = df.dropna(subset=['INTERSECTION'])

In [ ]:
def top_20_intersections(df):
    return df['INTERSECTION'].value_counts()[:20]

top_20_intersections(df)

## Feature Extraction: Street Numbers

In [ ]:
intersection = df_intersection['INTERSECTION']

In [ ]:
def get_street_numbers(df):
    street_numbers = intersection.str.extract(r'\W(\d+) ST', expand=False).astype(float)
    
    return street_numbers

In [ ]:
df_intersection.loc[:,'STREET NUMBER'] = get_street_numbers(df)

In [ ]:
between_1_14 = (df_intersection['STREET NUMBER']
                .between(1, 14, inclusive=True))

In [ ]:
df_intersection[between_1_14]['INTERSECTION'].value_counts()[:10]

In [ ]:
between_1_13 = df_intersection['STREET NUMBER'].between(1, 13, inclusive=True)
df_intersection[between_1_13]['INTERSECTION'].value_counts()[:10]

### Feature Engineering: Alphabet City

In [ ]:
def get_avenue_letters(df):
    avenue_letters = (df['INTERSECTION']
                      .str.extract(r'(AVENUE [A-D])(?:$|\s)', expand=False))
    
    return avenue_letters

In [ ]:
avenue_letters = intersection.str.extract(r'(AVENUE [A-D])(?:$|\s)', expand=False)

In [ ]:
avenue_letters.dropna()

In [ ]:
df_alphabet = df_intersection.loc[avenue_letters.notna(),:]
df_alphabet

In [ ]:
df_alphabet[df_alphabet['STREET NUMBER'].isna()]['INTERSECTION'].value_counts()

### Feature engineering: number of vehicles

In [ ]:
vehicle_type_codes = df[[
    'VEHICLE TYPE CODE 1',
    'VEHICLE TYPE CODE 2',
    'VEHICLE TYPE CODE 3',
    'VEHICLE TYPE CODE 4',
    'VEHICLE TYPE CODE 5'
]]

vehicle_type_codes.head()

In [ ]:
vehicle_type_codes.notna().head()

In [ ]:
def get_num_vehicles(df):
    vehicle_type_codes = df[[
    'VEHICLE TYPE CODE 1',
    'VEHICLE TYPE CODE 2',
    'VEHICLE TYPE CODE 3',
    'VEHICLE TYPE CODE 4',
    'VEHICLE TYPE CODE 5'
    ]]

    num_vehicles = vehicle_type_codes.notna().sum(axis=1)
    num_vehicles = num_vehicles.to_frame()
    num_vehicles.columns=['NUM VEHICLES']
    
    return num_vehicles

In [ ]:
num_vehicles = get_num_vehicles(df)

In [ ]:
num_vehicles.plot.hist()

### Join to original data frame 

In [ ]:
df = df.join(num_vehicles)
df.head()

In [ ]:
(df.groupby('INTERSECTION')['NUM VEHICLES']
   .agg(['count', 'mean'])
   .sort_values(by='count', ascending=False))[:20]

## What's next?

### Reproducible results

**Virtual environments**: 
- pip
- conda


**Hosting notebooks online**
* Jupyter notebooks on github
* Juptyer notebooks with Binder
    * Considerations for downloading data

### Back to the original problem

* Grouping deliveries by location
    * Heuristic rules
    * Clustering
* Towards a predictive model
    * Predicting best routes: again, what does "best" mean here?
    * What other data could we use?